## Testing google calendar API:

#### Installing the google calendar API:

In [2]:
# Installing 

# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

#### importing library:

In [163]:
import os.path
import pickle
from datetime import datetime, timedelta
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

#### Create a token file:

In [164]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar.events']

CREDENTIALS_FILE = '../client_secret.json'

def get_calendar_service():
   creds = None
   # The file token.pickle stores the user's access and refresh tokens, and is
   # created automatically when the authorization flow completes for the first
   # time.
   if os.path.exists('token.pickle'):
       with open('token.pickle', 'rb') as token:
           creds = pickle.load(token)
   # If there are no (valid) credentials available, let the user log in.
   if not creds or not creds.valid:
       if creds and creds.expired and creds.refresh_token:
           creds.refresh(Request())
       else:
           flow = InstalledAppFlow.from_client_secrets_file(
               CREDENTIALS_FILE, SCOPES)
           creds = flow.run_local_server(port=0)

       # Save the credentials for the next run
       with open('token.pickle', 'wb') as token:
           pickle.dump(creds, token)

   service = build('calendar', 'v3', credentials=creds)
   return service

#### Connect to user's calendar:

In [165]:
service = get_calendar_service()
service

#### Reading data to send the calendar:

In [23]:
fms_dates = pd.read_csv('../data/fms_dates.csv')

In [24]:
fms_dates

,League,#,Date,City
0,Argentina,1,2021-12-04,Mendoza
1,Argentina,2,2021-12-29,Buenos Aires
2,Argentina,3,2022-04-30,La Plata
3,Argentina,4,2022-06-18,Rosario
4,Argentina,5,2022-08-27,Corrientes
...,...,...,...,...
61,Peru,7,NaN,NaN
62,Peru,8,NaN,NaN
63,Peru,9,NaN,NaN
64,Peru,10,NaN,NaN


In [136]:
fms_dates[(fms_dates['League'] == 'Argentina') & (fms_dates['#'] == 5)]

,League,#,Date,City
4,Argentina,5,2022-08-27,Corrientes


In [131]:
start_date = fms_dates[(fms_dates['League'] == 'Argentina') & (fms_dates['#'] == 5)]['Date'].astype('datetime64[ns]').values[0]
new_start_date = pd.Timestamp(start_date)
new_start_date.isoformat()

'2022-08-27T00:00:00'

In [134]:
new_end_date = (new_start_date + timedelta(hours=5))
new_end_date.isoformat()

'2022-08-27T05:00:00'

In [166]:
def getting_fms_date(league, date_number):
    df_filtered = fms_dates[(fms_dates['League'] == league) & (fms_dates['#'] == date_number)]
    city = df_filtered['City'].values[0]
    
    start_date = pd.Timestamp(df_filtered['Date'].astype('datetime64[ns]').values[0])
    end_date = start_date + timedelta(hours=5)
    
    # time = EVENTUALLY WE WILL GET THE TIME
    
    return start_date.isoformat(), end_date.isoformat(), city #, time

In [157]:
getting_fms_date('Argentina', 2)

('2021-12-29T00:00:00', '2021-12-29T05:00:00', 'Buenos Aires')

#### Add an event:

In [167]:
def adding_an_event(league, date_number, city, start_datetime, end_datetime):
   service = get_calendar_service()

   event_result = service.events().insert(calendarId='primary',
       body={
           "summary": f'FMS {league} Jornada {date_number}',
           "description": f'FMS {league} en {city}. Jornada {date_number}',
           "start": {"dateTime": new_start_date.isoformat(), "timeZone": 'Europe/Madrid'},
           "end": {"dateTime": new_end_date.isoformat(), "timeZone": 'Europe/Madrid'},
           "reminders": {
                'useDefault': False,
                'overrides': [
                  {'method': 'email', 'minutes': 60},
                  {'method': 'popup', 'minutes': 10},
                            ],
                        },
            }
   ).execute()

   print("Created event")
   print("Id: ", event_result['id'])
   print("Summary: ", event_result['summary'])
   print("Starts at: ", event_result['start']['dateTime'])
   print("Ends at: ", event_result['end']['dateTime'])

#### Creating the event using the functions:

In [168]:
country = 'Argentina'
event_number = 5

event_list = getting_fms_date(country, event_number)
adding_an_event(country, event_number, event_list[2], event_list[0], event_list[1])

Created event
Id:  7a7epm4h5rka0fq2q2tpftvpcg
Summary:  FMS Argentina Jornada 5
Starts at:  2022-08-27T00:00:00+02:00
Ends at:  2022-08-27T05:00:00+02:00
